# Convolutional Neural Network

### Importing the libraries

In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [2]:
tf.__version__

'2.9.0'

## Part 1 - Data Preprocessing

훈련데이터 과적합 방지를 위한 이미지 증강(변형)

### Preprocessing the Training set

In [3]:
train_datagen = ImageDataGenerator(
          rescale = 1./255,
          shear_range = 0.2,
          zoom_range = 0.2,
          horizontal_flip = True)


In [4]:
training_set = train_datagen.flow_from_directory(
        'dataset/training_set',
        target_size = (64, 64),
        batch_size = 32,
        class_mode = 'binary')

Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

In [5]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory(
        'dataset/test_set',
        target_size = (64, 64),
        batch_size = 32,
        class_mode = 'binary')

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [6]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [7]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64,64,3]))

### Step 2 - Pooling

In [8]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [9]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [10]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [11]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 5 - Output Layer

In [12]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [16]:
cnn.compile(optimizer='adam',loss = 'binary_crossentropy', metrics=['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [17]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 25)
# batch_size=32 * 1epoch 당 250 단계(iteration) = 총 8000장 이미지 => 25epoch 반복
# ==> 총 8000*25 학습

Epoch 1/25
250/250 [==============================] - 433s 2s/step - loss: 0.6807 - accuracy: 0.5730 - val_loss: 0.6678 - val_accuracy: 0.6140
Epoch 2/25
250/250 [==============================] - 62s 248ms/step - loss: 0.6370 - accuracy: 0.6465 - val_loss: 0.6014 - val_accuracy: 0.6900
Epoch 3/25
250/250 [==============================] - 85s 341ms/step - loss: 0.5974 - accuracy: 0.6762 - val_loss: 0.5707 - val_accuracy: 0.7095
Epoch 4/25
250/250 [==============================] - 80s 321ms/step - loss: 0.5692 - accuracy: 0.7032 - val_loss: 0.5476 - val_accuracy: 0.7200
Epoch 5/25
250/250 [==============================] - 60s 239ms/step - loss: 0.5437 - accuracy: 0.7193 - val_loss: 0.5828 - val_accuracy: 0.6930
Epoch 6/25
250/250 [==============================] - 96s 385ms/step - loss: 0.5188 - accuracy: 0.7379 - val_loss: 0.5185 - val_accuracy: 0.7595
Epoch 7/25
250/250 [==============================] - 78s 313ms/step - loss: 0.5097 - accuracy: 0.7456 - val_loss: 0.5717 - val_accu

## Part 4 - Making a single prediction

In [25]:
import numpy as np
# from keras.preprocessing import image
from keras import utils

In [48]:
test_image = utils.load_img(
        'dataset/single_prediction/cat_or_dog_2.jpg'
        , target_size = (64,64)
        )

In [50]:
# 이미지 -> 2차원 배열로 변환
test_image = utils.img_to_array(test_image)

In [51]:
len(test_image)

64

In [52]:
# 초기 모델 배치사이즈 통일을 위함
# np.expand_dims(차원을 확장할 배열, 새로 추가될 차원의 위치) : 차원추가
test_image = np.expand_dims(test_image, axis = 0)

In [53]:
len(test_image[0])

64

In [43]:
result = cnn.predict(test_image / 225.0)

1/1 [==============================] - 0s 26ms/step


In [44]:
result

array([[0.9998683]], dtype=float32)

In [45]:
# 강아지1, 고양이0 확인
training_set.class_indices

{'cats': 0, 'dogs': 1}

In [46]:
if result[0][0]  > 0.5:
  prediction = 'dog'
else :
    prediction = 'cat'

In [47]:
print(prediction)

dog
